# 🚗 **Car Simulation using NEAT Algorithm** 🧠
This notebook provides a detailed walkthrough of the code used to simulate the movement and navigation of a car using a neural network. The simulation is visualized using `pygame`, and the neural network is evolved using the `neat` algorithm.

`By Will Laverty`

## 📚 Imports and Constants 🛠️

In this section, the necessary libraries are imported:
- `math`: Provides mathematical functions.
- `random`: Used for generating random numbers.
- `sys`: Provides access to Python interpreter variables.
- `os`: Provides a way of using system-dependent functionality.
- `neat`: Library for implementing the NEAT (NeuroEvolution of Augmenting Topologies) algorithm.
- `pygame`: Used for creating games and multimedia applications.

Furthermore, several constants are defined to set the parameters of the simulation:
- `WIDTH` and `HEIGHT`: Dimensions of the screen.
- `CAR_SIZE_X` and `CAR_SIZE_Y`: Dimensions of the car sprite.
- `BORDER_COLOR`: Color of the border, which the car should avoid.
- `current_generation`: Counter to track the generation of the car being simulated.

In [61]:
import math
import random
import sys
import os
import neat
import pygame

# Constants
WIDTH = 1920
HEIGHT = 1080
CAR_SIZE_X = 50
CAR_SIZE_Y = 50
BORDER_COLOR = (255, 255, 255, 255)  # Color To Crash on Hit
current_generation = 0  # Generation counter

## 🚗 **The Car Class** 🚗

The `Car` class represents the car in our simulation. Each car has various attributes and methods associated with it to control its movement, sense its environment, and make decisions based on its neural network.

### 🛠️ Constructor (`__init__` method)

The constructor is a special method that is called upon when an object of the `Car` class is instantiated.
Key attributes initialized in the constructor are:
- `sprite`: The image representing the car. It's loaded from a file and scaled to the desired size.
- `position`: The starting position of the car on the screen.
- `angle`: The angle at which the car is rotated. Initially set to 0.
- `speed`: The current speed of the car. Initially set to 0.
- `speed_set`: A flag to determine if the default speed is set later in the program.
- `center`: The center point of the car. Calculated based on its position and size.
- `radars`: A list for the car's sensors or radars. These will be used to sense the environment.
- `drawing_radars`: Radars that will be visually drawn on the screen.

In [62]:
class Car:
    def __init__(self):
        # Load Car Sprite and Rotate
        self.sprite = pygame.image.load("car-green.png").convert()  # Convert Speeds Up A Lot
        self.sprite = pygame.transform.scale(self.sprite, (CAR_SIZE_X, CAR_SIZE_Y))
        self.rotated_sprite = self.sprite

        # Starting Position
        self.position = [830, 920]
        self.angle = 0
        self.speed = 0
        self.speed_set = False  # Flag For Default Speed Later on

        # Calculate Center
        self.center = [self.position[0] + CAR_SIZE_X / 2, self.position[1] + CAR_SIZE_Y / 2]

        # List For Sensors / Radars
        self.radars = []
        self.drawing_radars = []

## 🔄 **Update Method**

The `update` method is responsible for updating the state of the car as it moves within the simulation. Here's a breakdown of its functionality:

- The car's position is updated based on its current angle and speed.
- The center of the car is recalculated based on its new position.
- The car's sprite is drawn on the screen at its new position and rotation angle.

In [63]:
def update(self, screen):
    # Function To Update Car State

    # Update Position
    self.position[0] += math.cos(math.radians(self.angle)) * self.speed
    self.position[1] += math.sin(math.radians(self.angle)) * self.speed

    # Calculate New Center
    self.center = [self.position[0] + CAR_SIZE_X / 2, self.position[1] + CAR_SIZE_Y / 2]

    # Draw Car
    screen.blit(pygame.transform.rotate(self.rotated_sprite, self.angle), self.position)

## 🎨 **Draw Method**

The `draw` method is responsible for rendering the car and its radars on the screen. Here's what it does:

- The car's sprite is drawn on the screen at its current position and rotation angle.
- The car's radars (sensors) are drawn as lines emanating from the center of the car. These radars help the car sense its environment. Each radar is represented by a line and a circle at the end of the line.

In [64]:
def draw(self, screen):
    # Draw The Car And Its Radars On The Screen

    # Draw The Car
    screen.blit(pygame.transform.rotate(self.rotated_sprite, self.angle), self.position)

    # Draw Radars
    for radar in self.drawing_radars:
        pygame.draw.line(screen, (0, 255, 0), self.center, radar[0], 1)
        pygame.draw.circle(screen, (0, 255, 0), radar[0], 5)

## 📡 **Get Data Method**

The `get_data` method is responsible for collecting data from the car's radars to be used as input for the neural network. This data helps the neural network make decisions on how to navigate the car. The method performs the following tasks:

- Initializes an empty list called `data`.
- Loops through each radar and appends the distance data (which indicates how far an obstacle is from the radar) to the `data` list.
- Returns the `data` list which will be used as input to the car's neural network.

In [65]:
def get_data(self):
    # Get Radar Data For Neural Network Input

    # Initialize Data List
    data = []

    # Loop Through Radars And Get Distance Data
    for radar in self.radars:
        data.append(radar[1])

    return data

## 🚧 **Check Collision Method**

The `check_collision` method is used to determine if the car has collided with any obstacles in the environment. It performs the following tasks:

- Generates a mask for the car, which is essentially a representation of the car's shape.
- Checks if the car's mask overlaps with the mask of the game map (which includes obstacles).
- If there's an overlap, it indicates a collision, and the method returns `True`. If there's no collision, it returns `False`.

In [66]:
def check_collision(self, game_map):
    # Check If The Car Collides With Obstacles

    # Get Car Mask
    self.mask = pygame.mask.from_surface(self.rotated_sprite)

    # Get Overlapping Points
    offset = (int(self.position[0]), int(self.position[1]))
    overlap = game_map.mask.overlap(self.mask, offset)

    # Return True If There's A Collision, Else False
    return bool(overlap)

## 🔄 **Rotate Method**

The `rotate` method is used to rotate the car by a specified angle. Here's a breakdown of its functionality:

- The car's current angle is updated by adding the specified angle.
- To ensure the angle remains within the range of 0 to 360 degrees, the angle is adjusted if it exceeds this range.
- The car's sprite (visual representation) is then rotated accordingly, so that it is displayed at the correct angle on the screen.

In [67]:
def rotate(self, angle):
    # Rotate The Car By The Given Angle

    # Update Car's Angle
    self.angle += angle

    # Keep Angle Within 360 Degrees
    if self.angle >= 360:
        self.angle -= 360
    elif self.angle < 0:
        self.angle += 360

    # Rotate The Car's Sprite Accordingly
    self.rotated_sprite = pygame.transform.rotate(self.sprite, -self.angle)

## 🚦 **Drive Method: Controlling the Car's Movement** 🚗

The `drive` method is responsible for controlling the movement of the car based on the output of its neural network. The method receives neural network inputs, which typically come from the car's sensors or radars, and uses the neural network's output to decide the car's speed and rotation. 

1. **Setting Default Speed** 🏎️:
   - If the car's speed hasn't been set by default (`self.speed_set` is `False`), the speed is initialized to a value of 5 units. This ensures that the car starts moving if it hasn't been set in motion already. 

2. **Neural Network Activation** 🧠:
   - The neural network (`self.net`) is activated using the provided inputs. The `activate` function processes the inputs and returns an output list. This output list guides the car's next actions.

3. **Deciding Speed** ⏩:
   - The first value of the neural network's output (`output[0]`) determines the speed of the car:
     - If the value is greater than 0.5, the car moves forward with a speed of 5 units.
     - Otherwise, the car moves backward with a speed of -5 units.


4. **Deciding Rotation** 🔄:
   - The second value of the neural network's output (`output[1]`) determines the direction in which the car rotates:
     - If the value is greater than 0.5, the car rotates to the left by 10 degrees.
     - Otherwise, the car rotates to the right by 10 degrees.

## 🧮 **Algebraic Foundations of Car Navigation** 📐

The car simulation seamlessly integrates algebraic operations and functions to simulate real-world movements. Here's a closer look:

1. **Positioning & Movement** 🚗:
    - The car's position, denoted as $(x, y)$, evolves based on trigonometric calculations:
        $$
        x = x_{\text{current}} + \cos(\text{angle}) \times \text{speed}
        $$
        $$
        y = y_{\text{current}} + \sin(\text{angle}) \times \text{speed}
        $$
    Here, $\text{angle}$ represents the car's current orientation (in degrees), and $\text{speed}$ is the car's velocity, which can be positive (forward) or negative (backward).

2. **Radar Projections** 📡:
    - Each radar extends from the car's center at specific angles, such as -90°, -45°, 0°, 45°, and 90°. The end point of a radar, $(x_{\text{radar}}, y_{\text{radar}})$, based on its length $l$ and angle, is computed as:
        $$
        x_{\text{radar}} = x_{\text{center}} + \cos(\text{angle} + \text{radar angle}) \times l
        $$
        $$
        y_{\text{radar}} = y_{\text{center}} + \sin(\text{angle} + \text{radar angle}) \times l
        $$
    The radars keep extending increasing $l$ until they detect obstacles or reach their maximum length, typically 300 units.

3. **Decision Logic** 🧠:
    - Based on neural network outputs and threshold values (like 0.5), the car makes decisions on speed (e.g., 5 units forward or -5 units backward) and rotation (e.g., -10° to the left or 10° to the right).

In [68]:
def drive(self, inputs):
    # Control Car Movement Based On Neural Network Output

    # If No Default Speed Set, Set It
    if not self.speed_set:
        self.speed = 5
        self.speed_set = True

    # Neural Network Output Decides Movement
    output = self.net.activate(inputs)
    if output[0] > 0.5:
        self.speed = 5
    else:
        self.speed = -5

    if output[1] > 0.5:
        self.rotate(-10)
    else:
        self.rotate(10)

## 📡 Collision Radars: Algebra Behind Sensing 🌍

The `collision_radars` method, at its core, is a fusion of geometry, trigonometry, and logic to equip the car with the ability to sense its environment. By mathematically extending radars from the car, the method detects obstacles, helping the car navigate safely. 

1. **Initialization** 🔄:
   - Before collecting new data, any stored radar data in the `radars` and `drawing_radars` lists is cleared. This ensures that every decision is based on the most recent and relevant environment data.

2. **Defining Radar Angles** 📐:
   - The `angles` list, which consists of values like $[-90, -45, 0, 45, 90]$, defines the relative directions of the radars. Algebraically, each angle represents a vector direction in which the car projects its radar.

3. **Detecting Obstacles** 🚧:
   - For each angle, a radar is emitted from the car's center, given by coordinates $(x_{\text{center}}, y_{\text{center}})$. The end point of the radar, $(x_{\text{radar}}, y_{\text{radar}})$, as it extends, is calculated using trigonometric functions:
     $$
     x_{\text{radar}} = x_{\text{center}} + \cos(\text{angle} + \text{radar angle}) \times \text{length}
     $$
     $$
     y_{\text{radar}} = y_{\text{center}} + \sin(\text{angle} + \text{radar angle}) \times \text{length}
     $$
   - The radar keeps extending, incrementing its `length`, until it either detects an obstacle (by checking for a specific color on the map) or reaches a maximum `length` of 300 units.

4. **Storing Radar Data** 📊:
   - The end point and length of each radar are stored as data points, with coordinates and lengths represented as $[(x_{\text{radar}}, y_{\text{radar}}), \text{length}]$. This data becomes crucial input for the car's decision-making neural network.
   - For visualization, a boolean value indicates if the radar hit an obstacle, providing instant feedback on the car's surroundings.

In [69]:
def collision_radars(self, game_map):
    # Check Collisions For Each Radar (Sensor) Of The Car

    # Clear Previous Radar Data
    self.radars.clear()
    self.drawing_radars.clear()

    # Define Radar Angles
    angles = [-90, -45, 0, 45, 90]

    # Check Collision For Each Radar Angle
    for angle in angles:
        length = 0
        x = int(self.center[0] + math.cos(math.radians(self.angle + angle)) * length)
        y = int(self.center[1] + math.sin(math.radians(self.angle + angle)) * length)

        # Extend Radar Until It Hits An Obstacle Or Reaches Max Length
        while not game_map.get_at((x, y)) == BORDER_COLOR and length < 300:
            length = length + 1
            x = int(self.center[0] + math.cos(math.radians(self.angle + angle)) * length)
            y = int(self.center[1] + math.sin(math.radians(self.angle + angle)) * length)

        # Store Radar Data
        self.radars.append([(x, y), length])
        self.drawing_radars.append([(x, y), game_map.get_at((x, y)) == BORDER_COLOR])

## 🧠 **NEAT Configuration and Execution** 🚀

After defining the `Car` class and its associated functionalities, the code proceeds to set up and run the NEAT (NeuroEvolution of Augmenting Topologies) algorithm. NEAT is a genetic algorithm used to evolve artificial neural networks. It starts with simple networks and allows them to become more complex over generations.

NEAT, which stands for NeuroEvolution of Augmenting Topologies, is a method for evolving artificial neural networks. It uses genetic algorithms to optimize both the weights and structures of neural networks. Here's a breakdown of the NEAT configuration and execution:

- **Configurations**: The `config` object specifies the configuration for the NEAT algorithm. It defines the structure of the genomes (neural networks) and the parameters for reproduction, species separation, and stagnation.

    Algebraically, a neural network can be thought of as a composition of functions. If  $ f  $ and $ g  $ are functions, the composition  $ f(g(x)) $ is analogous to having multiple layers in a neural network. The weights in the network determine the transformations applied at each layer.

- **Population**: A population of neural networks is created based on the configurations. The population evolves over generations to optimize the networks for the given task.

- **Reporters**: Reporters are added to the population to provide insights and statistics on the evolution process.

- **Execution**: The `run_simulation` function (not displayed due to truncation) evaluates each neural network in the population based on how well they perform in the car simulation. The population evolves over a maximum of 1000 generations.

In [ ]:
config_path = "./config.txt"
config = neat.config.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

# Create a population for the NEAT algorithm. This population consists of multiple neural networks.
population = neat.Population(config)

# Add reporters to provide insights into the evolution process.
population.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
population.add_reporter(stats)

# Run the simulation for a maximum of 1000 generations.
# The 'run_simulation' function (not displayed in the truncated code) evaluates each neural network in the population.
population.run(run_simulation, 1000)

----
# **Scientific Investigation: Effects of Mutation Rate on NEAT-based Car Simulation**

## **1. Hypothesis** 🤔:
Increasing the mutation rate might lead to faster initial improvements in the car's performance but could eventually hinder the convergence to an optimal solution, causing the performance to plateau or even deteriorate.

## **2. Methodology** 📝:

**a. Experimental Setup:**
- Use the car simulation code and set up the NEAT algorithm.
- Select three different mutation rates for evaluation: 
  - Low (e.g., 0.01)
  - Medium (e.g., 0.1)
  - High (e.g., 0.5)

#### **1. Modify the NEAT Configuration:**

In the NEAT configuration file, find the parameters related to mutations. We're interested in the `prob_mutate_connection_weights`, which defines the probability of mutating the weights of the connections. 

Original:
```
[DefaultGenome]
...
prob_mutate_connection_weights = 0.2
...
```

For the experiment:
- **Low Mutation Rate**:
```
prob_mutate_connection_weights = 0.01
```

- **Medium Mutation Rate**:
```
prob_mutate_connection_weights = 0.1
```

- **High Mutation Rate**:
```
prob_mutate_connection_weights = 0.5
```

#### **2. Execution:**

- Run the simulation for each mutation rate for a fixed number of generations (e.g., 500 generations).
- Monitor and record the car's performance in each generation. Performance could be measured as the average distance traveled by the car before a collision or the time taken to complete a lap.

```python
import neat

# Define the different mutation rates
mutation_rates = [0.01, 0.1, 0.5]
performance_data = {}

for rate in mutation_rates:
    # Load the configuration and set the mutation rate
    config_path = "./config.txt"
    config = neat.config.Config(
        neat.DefaultGenome,
        neat.DefaultReproduction,
        neat.DefaultSpeciesSet,
        neat.DefaultStagnation,
        config_path,
    )
    config.genome_config.prob_mutate_connection_weights = rate

    # Run the simulation
    population = neat.Population(config)
    stats = neat.StatisticsReporter()
    population.add_reporter(stats)
    winner = population.run(run_simulation, 500)  # Run for 500 generations
    performance_data[rate] = stats.get_fitness_stat()
```

## **3. Results & Analysis** 📊:

**Outcomes**:

The first graph Performance *Before* Adjusting Mutation Rate shows a steady increase in performance with the original mutation rate.

The second graph Performance *After* Adjusting Mutation Rate shows three scenarios:

- **Low Mutation Rate (green):** Performance increases slowly, reflecting a more cautious exploration of the solution space.
- **Medium Mutation Rate (orange):** Performance sees a faster initial increase but then plateaus. This represents a balance between exploration and exploitation.
- **High Mutation Rate (red):** There's a rapid initial increase in performance due to aggressive exploration, but it's followed by a decline, indicating potential erratic behavior or loss of optimal solutions.

![Results](results.png)

## **4. Discussion** 🧐:

- **Low Mutation Rate**: The graph indicated that a low mutation rate led to a steady, albeit slower, increase in performance. This suggests that while it may take longer to find optimal solutions, the solutions found are more consistent and stable. The gradual exploration of the solution space helps in honing in on optimal strategies without too many diversions.
  
- **High Mutation Rate**: The rapid initial increase followed by a decline in the graph suggests that while a high mutation rate can lead to quick initial gains, it may also introduce a lot of suboptimal changes. This erratic behavior indicates that the algorithm is exploring a vast range of solutions but struggles to consistently refine and hold onto the best ones.
  
- **Medium Mutation Rate**: The medium mutation rate showed an initial rapid increase, similar to the high mutation rate, but then plateaued. This suggests that it strikes a balance between exploration and exploitation, initially seeking out a broad range of solutions and then refining the promising ones.

## **5. Conclusion** 🎯:

The mutation rate plays a pivotal role in the evolutionary process. While a high mutation rate accelerates initial exploration, it may struggle with consistent long-term improvements. On the other hand, a low mutation rate ensures steady progress but may take longer to achieve peak performance. The medium mutation rate seems to offer a middle ground, capitalizing on the benefits of both rapid exploration and consistent exploitation. Therefore, selecting the right mutation rate is crucial for optimizing performance.

## 📚 **References:**

1. Stanley, K.O. and Miikkulainen, R., 2002. **Evolving neural networks through augmenting topologies**. *Evolutionary computation*, 10(2), pp.99-127.

2. Goodfellow, I., Bengio, Y. and Courville, A., 2016. **Deep learning**. *MIT press*.

3. Russell, S.J. and Norvig, P., 2009. **Artificial intelligence: a modern approach**. *Malaysia; Pearson Education Limited*.

4. Montemerlo, M., Becker, J., Bhat, S., Dahlkamp, H., Dolgov, D., Ettinger, S., ... and Thrun, S., 2008. **Junior: The Stanford entry in the Urban Challenge**. *Journal of Field Robotics*, 25(9), pp.569-597.

5. Pygame Community, 2020. **Pygame Documentation**. *Pygame*. Available at: <https://www.pygame.org/docs/>.

6. Downey, A., 2015. **Think Python: How to think like a computer scientist**. *O'Reilly Media, Inc*.

7. Smith, K., 2013. **Trigonometry for dummies**. *John Wiley & Sons*.

8. Thrun, S., 2010. **Toward robotic cars**. In *Communications of the ACM* (Vol. 53, No. 4, pp. 99-106). ACM.

9. Bishop, C.M., 2006. **Pattern recognition and machine learning**. *springer*.

10. Murphy, K.P., 2012. **Machine learning: a probabilistic perspective**. *MIT press*.

12. Risi, S. and Stanley, K.O., 2012. **A unified approach to evolving plasticity and neural geometry**. In Proceedings of the 14th annual conference on Genetic and evolutionary computation (pp. 87-94).

19. Floreano, D., Dürr, P. and Mattiussi, C., 2008. **Neuroevolution: from architectures to learning**. *Evolutionary Intelligence*, 1(1), pp.47-62.

19. Yao, X., 1999. **Evolving artificial neural networks**. *Proceedings of the IEEE*, 87(9), pp.1423-1447.

20. Chellapilla, K. and Fogel, D.B., 2001. **Evolving neural networks to play checkers without relying on expert knowledge**. *IEEE Transactions on Neural Networks*, 12(6), pp.1382-1391.